# grafo das palavras e isso das empresas

In [1]:
import networkx as nx
import plotly.graph_objects as go
import pandas as pd

# Sample data (You should replace this with your actual dataframe)
df = pd.read_parquet('data05.parquet')
df["keywords"] = df["keywords"].map(lambda dic: {key: dic[key] for key in dic.keys() if dic[key] is not None and dic[key]["filter"] > 0.05})

# Using the first row as an example
linha = 1
data = df["keywords"].iloc[linha]
company = df["keywords"].index[linha]

# Create the graph
G = nx.Graph()

# Add nodes to the graph with attributes
for word, attributes in data.items():
    G.add_node(word, **attributes)
    G.add_edge(company, word)
    #G.nodes[word]["info"] = f"This is additional information about {word}."

# Node positions
pos = nx.spring_layout(G, seed=42)

# Lists for node positions and info
node_x = []
node_y = []
node_color = []
node_text = []
node_hovertext = []
custom_data = []

for node in G.nodes:
    if node == company:
        x, y = 0, 0
        node_x.append(x)
        node_y.append(y)
        node_text.append(node)
        node_hovertext.append(f"Company: {node}")
        node_color.append("black")
        custom_data.append("custom data for company allez allez")
        continue
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    node_text.append(node)
    
    # Color from red (-1) to green (1)
    sentiment = G.nodes[node]["sentiment"]
    if sentiment < -0.5:
        node_color.append("rgb(204, 0, 0)") #triste
    elif sentiment < -0.05:
        node_color.append("rgb(239, 83, 80)") # tristonho
    elif sentiment < 0.1:
        node_color.append("rgb(204, 204, 204)") # neutro
    elif sentiment < 0.55:
        node_color.append("rgb(102, 187, 106)") # feliz
    else:
        node_color.append("rgb(0, 200, 81)") # iupi

    last_time_said = max((key for key, value in G.nodes[node]["date"].items() if value is not None))
    node_hovertext.append(
        f"""Word: {node}
        <br>Count: {int(G.nodes[node]['count'])}
        <br>Last said: {str(last_time_said)[:4] + "-" + str(last_time_said)[-2:6]}"""
    )

    # SOURCE FOR CUSTOM DATA
    source = G.nodes[node]["source"]
    source_data = ""
    for key in source.keys():
        if source[key] is not None:
            source_data += f"<li>{key}: {int(source[key])}</li>"
    # WEBSITES FOR CUSTOM DATA
    websites = G.nodes[node]["news"]
    websites_data = ""
    for website in websites:
        website = website.replace("/wayback/", "/noFrame/replay/")
        websites_data += f"<li><a href='{website}' target='_blank'>{website}</a></li>"

    custom_data.append(f"""
        <h2>Information on {node}:</h2>
        <p>Count: {G.nodes[node]['count']}</p>
        <p>Sentiment: {round(G.nodes[node]["sentiment"], 3)}</p>
        <p>Source:</p>
        <ul>
        {source_data}
        </ul>
        <div id="scrollable-content">
            <ul>
                {websites_data}
            </ul>
        </div>
        <p> iupi </p>
        """)

# Create the Plotly figure
fig = go.Figure()

# Draw edges
edge_x = []
edge_y = []

for edge in G.edges:
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

fig.add_trace(
    go.Scatter(
        x=edge_x,
        y=edge_y,
        line=dict(width=1, color="gray"),
        hoverinfo="none",
        mode="lines"
    )
)

# Draw nodes
fig.add_trace(
    go.Scatter(
        x=node_x,
        y=node_y,
        mode="markers+text",
        text=node_text,
        #marker=node_text,
        hovertext=node_hovertext,
        marker=dict(
            color=node_color,
            size=20,
            line=dict(color="black", width=1)
        ),
        hoverinfo="text",
        customdata=custom_data
        
    )
)

fig.update_layout(
    showlegend=False,
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    margin=dict(l=0, r=0, t=0, b=0)  # Remove all margins
)

# Generate the HTML with full document structure
html_code = fig.to_html(include_plotlyjs='inline', full_html=True)

# Modify the generated HTML to include the side panel and additional styles/scripts
additional_html = """
<style>
    body, html {
        height: 100%;
        margin: 0;
        font-family: Arial, sans-serif;
        overflow: hidden;
    }
    #graph {
        position: absolute;
        top: 0;
        left: 0;
        right: 0;
        bottom: 0;
        z-index: 1; /* Ensure the graph is behind the panel */
    }
    #info-panel {
        position: absolute;
        right: 0; /* Default to right */
        top: 0;
        width: 300px; /* Fixed width of the panel */
        height: 100vh; /* Full height of container */
        background-color: #f4f4f9;
        box-shadow: -2px 0 5px rgba(0,0,0,0.1);
        transform: translateX(100%); /* Hide initially */
        transition: transform 0.3s ease; /* Smooth slide in */
        z-index: 2; /* Panel above the graph */
        display: flex;
        flex-direction: column; /* Arrange children vertically */
    }
    #info-panel.open {
        transform: translateX(0); /* Slide in the panel */
    }
    .close-button {
        background-color: #ff4c4c;
        color: white;
        border: none;
        padding: 10px;
        cursor: pointer;
        float: right;
        margin-bottom: 0px;
    }

    #scrollable-content {
        flex: 1; /* Take remaining vertical space */
        max-height: 200px; /* Adjust height as needed */
        overflow-y: auto; /* Enable vertical scrolling */
        padding: 10px; /* Padding for inner content */
        background-color: #ffffff; /* Background color */
        border: 1px solid #ddd; /* Border for the scrollable area */
        box-shadow: inset 0 0 5px rgba(0,0,0,0.1); /* Inner shadow */
        margin-top: 10px; /* Spacing from the title */
        }
</style>

<div id="info-panel">
    <button class="close-button" onclick="closePanel()">Close</button>
    <p id="node-info">Click a node to view details</p>
</div>

<script>
    function closePanel() {
        var panel = document.getElementById('info-panel');
        panel.classList.remove('open');
        // Manter a posição original do painel
        panel.style.left = ''; // Limpa a posição da esquerda
        panel.style.right = ''; // Limpa a posição da direita
    }

    document.addEventListener('DOMContentLoaded', function() {
        var plotDiv = document.querySelector('.plotly-graph-div');
        
        if (plotDiv) {
            plotDiv.on('plotly_click', function(data) {
                if (data.points.length > 0) {
                    var point = data.points[0];

                    // Accessing custom data
                    var customData = point.customdata; // Access custom data here
                    var additionalInfo = customData; // Format the custom data for display

                    document.getElementById('node-info').innerHTML = additionalInfo; // Use custom data for the panel

                    // Determine mouse click position for panel placement
                    var mouseX = data.event.clientX; // Get mouse X position
                    var panel = document.getElementById('info-panel');

                    // Adjust panel position based on mouse click
                    if (mouseX < window.innerWidth / 2) {
                        panel.style.right = '0'; // Show on the right
                        panel.style.left = 'auto'; // Clear left position
                    } else {
                        panel.style.left = '0'; // Show on the left
                        panel.style.right = 'auto'; // Clear right position
                    }

                    // Show the panel
                    panel.classList.add('open'); // Show panel on click
                }
            });
        } else {
            console.error("Graph div not found.");
        }
    });

</script>
"""

# Combine the Plotly HTML and the additional HTML
html_code = html_code.replace("</body>", additional_html + "</body>")

# Save to HTML file
with open("interactive_graph.html", "w") as file:
    file.write(html_code)

print("HTML file with interactive graph and side panel created as 'interactive_graph.html'.")

HTML file with interactive graph and side panel created as 'interactive_graph.html'.


---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---

In [2]:
import networkx as nx
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Sample data (You should replace this with your actual dataframe)
df = pd.read_parquet('data05.parquet')
df["keywords"] = df["keywords"].map(lambda dic: {key: dic[key] for key in dic.keys() if dic[key] is not None and dic[key]["filter"] > 0.05})

# Using the first row as an example
linha = 1
data = df["keywords"].iloc[linha]
company = df["keywords"].index[linha]

# Create the graph
G = nx.Graph()

# Add nodes to the graph with attributes
for word, attributes in data.items():
    G.add_node(word, **attributes)
    G.add_edge(company, word)

# Node positions
pos = nx.spring_layout(G, seed=42)

# Lists for node positions and info
node_x = []
node_y = []
node_color = []
node_text = []
node_hovertext = []
custom_data = []
node_size = []

for node in G.nodes:
    if node == company:
        x, y = 0, 0
        node_x.append(x)
        node_y.append(y)
        node_text.append(node)
        node_hovertext.append(f"Company: {node}")
        node_color.append("black")
        custom_data.append("custom data for company allez allez")
        continue
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    node_text.append(node)
    node_size.append(np.log(G.nodes[node]["weight"]))
    
    # Color from red (-1) to green (1)
    sentiment = G.nodes[node]["sentiment"]
    if sentiment < -0.5:
        node_color.append("rgb(204, 0, 0)") #triste
    elif sentiment < -0.05:
        node_color.append("rgb(239, 83, 80)") # tristonho
    elif sentiment < 0.1:
        node_color.append("rgb(204, 204, 204)") # neutro
    elif sentiment < 0.55:
        node_color.append("rgb(102, 187, 106)") # feliz
    else:
        node_color.append("rgb(0, 200, 81)") # iupi

    last_time_said = max((key for key, value in G.nodes[node]["date"].items() if value is not None))
    node_hovertext.append(
        f"""Word: {node}
        <br>Count: {int(G.nodes[node]['count'])}
        <br>Last said: {str(last_time_said)[:4] + "-" + str(last_time_said)[-2:6]}"""
    )

    # SOURCE FOR CUSTOM DATA
    source = G.nodes[node]["source"]
    source_data = ""
    for key in source.keys():
        if source[key] is not None:
            source_data += f"<li>{key}: {int(source[key])}</li>"
    # WEBSITES FOR CUSTOM DATA
    websites = G.nodes[node]["news"]
    websites_data = ""
    for website in websites:
        website = website.replace("/wayback/", "/noFrame/replay/")
        websites_data += f"<li><a href='{website}' target='_blank'>{website}</a></li>"

    custom_data.append(f"""
        <h2>Information on {node}:</h2>
        <p>Count: {G.nodes[node]['count']}</p>
        <p>Sentiment: {round(G.nodes[node]["sentiment"], 3)}</p>
        <p>Source:</p>
        <ul>
        {source_data}
        </ul>
        <div id="scrollable-content">
            <ul>
                {websites_data}
            </ul>
        </div>
        <p> iupi </p>
        """)

# Create the Plotly figure
fig = go.Figure()

# Draw edges
edge_x = []
edge_y = []

for edge in G.edges:
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

fig.add_trace(
    go.Scatter(
        x=edge_x,
        y=edge_y,
        line=dict(width=1, color="gray"),
        hoverinfo="none",
        mode="lines"
    )
)

# Draw nodes
fig.add_trace(
    go.Scatter(
        x=node_x,
        y=node_y,
        mode="markers+text",
        text=node_text,
        #marker=node_text,
        hovertext=node_hovertext,
        marker=dict(
            color=node_color,
            size=[w*10 for w in node_size],
            line=dict(color="black", width=1)
        ),
        hoverinfo="text",
        customdata=custom_data
        
    )
)

fig.update_layout(
    showlegend=False,
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    margin=dict(l=0, r=0, t=0, b=0)  # Remove all margins
)

# Generate the HTML with full document structure
html_code = fig.to_html(include_plotlyjs='inline', full_html=True)

# Modify the generated HTML to include the side panel and additional styles/scripts
additional_html = """
<style>
    body, html {
        height: 100%;
        margin: 0;
        font-family: Arial, sans-serif;
        overflow: hidden;
    }
    #graph {
        position: absolute;
        top: 0;
        left: 0;
        right: 0;
        bottom: 0;
        z-index: 1; /* Ensure the graph is behind the panel */
    }
    #info-panel {
        position: absolute;
        right: 0; /* Default to right */
        top: 0;
        width: 300px; /* Fixed width of the panel */
        height: 100vh; /* Full height of container */
        background-color: #f4f4f9;
        box-shadow: -2px 0 5px rgba(0,0,0,0.1);
        transform: translateX(100%); /* Hide initially */
        transition: transform 0.3s ease; /* Smooth slide in */
        z-index: 2; /* Panel above the graph */
        display: flex;
        flex-direction: column; /* Arrange children vertically */
    }
    #info-panel.open {
        transform: translateX(0); /* Slide in the panel */
    }
    .close-button {
        background-color: #ff4c4c;
        color: white;
        border: none;
        padding: 10px;
        cursor: pointer;
        float: right;
        margin-bottom: 0px;
    }

    #scrollable-content {
        flex: 1; /* Take remaining vertical space */
        max-height: 200px; /* Adjust height as needed */
        overflow-y: auto; /* Enable vertical scrolling */
        padding: 10px; /* Padding for inner content */
        background-color: #ffffff; /* Background color */
        border: 1px solid #ddd; /* Border for the scrollable area */
        box-shadow: inset 0 0 5px rgba(0,0,0,0.1); /* Inner shadow */
        margin-top: 10px; /* Spacing from the title */
        }
</style>

<div id="info-panel">
    <button class="close-button" onclick="closePanel()">Close</button>
    <p id="node-info">Click a node to view details</p>
</div>

<script>
    function closePanel() {
        var panel = document.getElementById('info-panel');
        panel.classList.remove('open');
        // Manter a posição original do painel
        panel.style.left = ''; // Limpa a posição da esquerda
        panel.style.right = ''; // Limpa a posição da direita
    }

    document.addEventListener('DOMContentLoaded', function() {
        var plotDiv = document.querySelector('.plotly-graph-div');
        
        if (plotDiv) {
            plotDiv.on('plotly_click', function(data) {
                if (data.points.length > 0) {
                    var point = data.points[0];

                    // Accessing custom data
                    var customData = point.customdata; // Access custom data here
                    var additionalInfo = customData; // Format the custom data for display

                    document.getElementById('node-info').innerHTML = additionalInfo; // Use custom data for the panel

                    // Determine mouse click position for panel placement
                    var mouseX = data.event.clientX; // Get mouse X position
                    var panel = document.getElementById('info-panel');

                    // Adjust panel position based on mouse click
                    if (mouseX < window.innerWidth / 2) {
                        panel.style.right = '0'; // Show on the right
                        panel.style.left = 'auto'; // Clear left position
                    } else {
                        panel.style.left = '0'; // Show on the left
                        panel.style.right = 'auto'; // Clear right position
                    }

                    // Show the panel
                    panel.classList.add('open'); // Show panel on click
                }
            });
        } else {
            console.error("Graph div not found.");
        }
    });

</script>
"""

# Combine the Plotly HTML and the additional HTML
html_code = html_code.replace("</body>", additional_html + "</body>")

# Save to HTML file
with open("interactive_graph.html", "w") as file:
    file.write(html_code)

print("HTML file with interactive graph and side panel created as 'interactive_graph.html'.")

HTML file with interactive graph and side panel created as 'interactive_graph.html'.


In [3]:
import networkx as nx
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import random
random.seed(42)

# Sample data (Replace this with your actual dataframe)
df = pd.read_parquet('data05.parquet')
df["keywords"] = df["keywords"].map(lambda dic: {key: dic[key] for key in dic.keys() if dic[key] is not None and dic[key]["filter"] > 0.05})

# Using the first row as an example
linha = 1
data = df["keywords"].iloc[linha]
company = df["keywords"].index[linha]

# Create the graph
G = nx.Graph()

# Add nodes to the graph with attributes
for word, attributes in data.items():
    G.add_node(word, **attributes)
    G.add_edge(company, word)

# Node positions
pos = nx.spring_layout(G, seed=42)

# Define concentric circle distances based on recency
date_distances = {
    "level1": 1,  # Most recent nodes
    "level2": 1.5,  # Most recent nodes
    "level3": 2,       # Recently mentioned nodes
    "level4": 2.5,          # Moderately recent
    "level5": 3,     # Moderately recent
    "level8": 1
}

# Lists for node positions and info
node_x = []
node_y = []
node_color = []
node_text = []
node_hovertext = []
custom_data = []
node_size = []
node_distances = []  # Store distances for scaling node positions

# Populate node information
for node in G.nodes:
    if node == company:
        x, y = 0, 0
        node_x.append(x)
        node_y.append(y)
        node_color.append("black")
        node_text.append(node)
        node_hovertext.append(f"Company: {node}")
        custom_data.append("custom data for company allez allez")
        node_size.append(10)
        node_distances.append(0)  # Center node distance
        continue
    
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    node_text.append(node)
    node_size.append(np.log(G.nodes[node]["weight"]))
    
    # Color from red (-1) to green (1)
    sentiment = G.nodes[node]["sentiment"]
    if sentiment < -0.5:
        node_color.append("rgb(204, 0, 0)")
    elif sentiment < -0.05:
        node_color.append("rgb(239, 83, 80)")
    elif sentiment < 0.1:
        node_color.append("rgb(204, 204, 204)")
    elif sentiment < 0.55:
        node_color.append("rgb(102, 187, 106)")
    else:
        node_color.append("rgb(0, 200, 81)")

    last_time_said = max(int(key) for key, value in G.nodes[node]["date"].items() if value is not None)
    node_hovertext.append(
        f"""Word: {node}
        <br>Count: {int(G.nodes[node]['count'])}
        <br>Last said: {str(last_time_said)[:4] + "-" + str(last_time_said)[-2:6]}"""
    )

    # Determine distance category
    years_ago = 202012 - last_time_said
    if True:
        node_distances.append(date_distances["level8"])
    elif years_ago < 99 and random.random() <= 0.4: #ultimo semestre
        node_distances.append(date_distances["level4"])
    elif years_ago < 3 and random.random() <= 0.6: #ultimo semestre
        node_distances.append(date_distances["level2"])
    elif years_ago < 8: #ainda em 2020
        node_distances.append(date_distances["level3"])
    elif years_ago < 99: #2019 e afins
        node_distances.append(date_distances["level4"])
    else: # 2019 e afins
        node_distances.append(date_distances["level5"])

    # SOURCE FOR CUSTOM DATA
    source = G.nodes[node]["source"]
    source_data = ""
    for key in source.keys():
        if source[key] is not None:
            source_data += f"<li>{key}: {int(source[key])}</li>"
    
    # WEBSITES FOR CUSTOM DATA
    websites = G.nodes[node]["news"]
    websites_data = ""
    for website in websites:
        website = website.replace("/wayback/", "/noFrame/replay/")
        websites_data += f"<li><a href='{website}' target='_blank'>{website}</a></li>"

    custom_data.append(f"""
        <h2>Information on {node}:</h2>
        <p>Count: {int(G.nodes[node]['count'])}</p>
        <p>Sentiment: {round(G.nodes[node]["sentiment"], 3)}</p>
        <p>Source:</p>
        <ul>
        {source_data}
        </ul>
        <div id="scrollable-content">
            <ul>
                {websites_data}
            </ul>
        </div>
        <p> iupi </p>
        """)



# Scale node positions by distance
node_x_scaled = []
node_y_scaled = []

for i, (x, y) in enumerate(zip(node_x, node_y)):
    scale_factor = node_distances[i]
    node_x_scaled.append(x * scale_factor)
    node_y_scaled.append(y * scale_factor)

# Create the Plotly figure
fig = go.Figure()

# Draw concentric circles
for distance in date_distances.values():
    theta = np.linspace(0, 2 * np.pi, 100)
    x_circle = distance * np.cos(theta)
    y_circle = distance * np.sin(theta)
    
    fig.add_trace(
        go.Scatter(
            x=x_circle,
            y=y_circle,
            mode="markers",
            hoverinfo="skip",
            marker=dict(
                color="lightgray",
                size=4
            ),
        )
    )



# Draw nodes
fig.add_trace(
    go.Scatter(
        x=node_x_scaled,
        y=node_y_scaled,
        mode="markers+text",
        text=node_text,
        hovertext=node_hovertext,
        marker=dict(
            color=node_color,
            size=[w * 1 for w in node_size],
            line=dict(color="black", width=1)
        ),
        hoverinfo="text",
        customdata=custom_data
    )
)

fig.update_layout(
    showlegend=False,
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    margin=dict(l=0, r=0, t=0, b=0)  # Remove all margins
)

# Generate the HTML with full document structure
html_code = fig.to_html(include_plotlyjs='inline', full_html=True)

# Modify the generated HTML to include the side panel and additional styles/scripts
additional_html = """
<style>
    body, html {
        height: 100%;
        margin: 0;
        font-family: Arial, sans-serif;
        overflow: hidden;
    }
    #graph {
        position: absolute;
        top: 0;
        left: 0;
        right: 0;
        bottom: 0;
        z-index: 1; /* Ensure the graph is behind the panel */
    }
    #info-panel {
        position: absolute;
        right: 0; /* Default to right */
        top: 0;
        width: 300px; /* Fixed width of the panel */
        height: 100vh; /* Full height of container */
        background-color: #f4f4f9;
        box-shadow: -2px 0 5px rgba(0,0,0,0.1);
        transform: translateX(100%); /* Hide initially */
        transition: transform 0.3s ease; /* Smooth slide in */
        z-index: 2; /* Panel above the graph */
        display: flex;
        flex-direction: column; /* Arrange children vertically */
    }
    #info-panel.open {
        transform: translateX(0); /* Slide in the panel */
    }
    .close-button {
        background-color: #ff4c4c;
        color: white;
        border: none;
        padding: 10px;
        cursor: pointer;
        float: right;
        margin-bottom: 0px;
    }

    #scrollable-content {
        flex: 1; /* Take remaining vertical space */
        max-height: 200px; /* Adjust height as needed */
        overflow-y: auto; /* Enable vertical scrolling */
        padding: 10px; /* Padding for inner content */
        background-color: #ffffff; /* Background color */
        border: 1px solid #ddd; /* Border for the scrollable area */
        box-shadow: inset 0 0 5px rgba(0,0,0,0.1); /* Inner shadow */
        margin-top: 10px; /* Spacing from the title */
        }
</style>

<div id="info-panel">
    <button class="close-button" onclick="closePanel()">Close</button>
    <p id="node-info">Click a node to view details</p>
</div>

<script>
    function closePanel() {
        var panel = document.getElementById('info-panel');
        panel.classList.remove('open');
        // Manter a posição original do painel
        panel.style.left = ''; // Limpa a posição da esquerda
        panel.style.right = ''; // Limpa a posição da direita
    }

    document.addEventListener('DOMContentLoaded', function() {
        var plotDiv = document.querySelector('.plotly-graph-div');
        
        if (plotDiv) {
            plotDiv.on('plotly_click', function(data) {
                if (data.points.length > 0) {
                    var point = data.points[0];

                    // Accessing custom data
                    var customData = point.customdata; // Access custom data here
                    var additionalInfo = customData; // Format the custom data for display

                    document.getElementById('node-info').innerHTML = additionalInfo; // Use custom data for the panel

                    // Determine mouse click position for panel placement
                    var mouseX = data.event.clientX; // Get mouse X position
                    var panel = document.getElementById('info-panel');

                    // Adjust panel position based on mouse click
                    if (mouseX < window.innerWidth / 2) {
                        panel.style.right = '0'; // Show on the right
                        panel.style.left = 'auto'; // Clear left position
                    } else {
                        panel.style.left = '0'; // Show on the left
                        panel.style.right = 'auto'; // Clear right position
                    }

                    // Show the panel
                    panel.classList.add('open'); // Show panel on click
                }
            });
        } else {
            console.error("Graph div not found.");
        }
    });

</script>
"""

# Combine the Plotly HTML and the additional HTML
final_html = html_code.replace("</body>", additional_html + "</body>")

# Save the HTML file
with open('graph_with_concentric_circles.html', 'w') as f:
    f.write(final_html)

print("HTML file with concentric circles created successfully.")


HTML file with concentric circles created successfully.


In [4]:
import networkx as nx
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import random
random.seed(42)

# Sample data (Replace this with your actual dataframe)
df = pd.read_parquet('data05.parquet')
df["keywords"] = df["keywords"].map(lambda dic: {key: dic[key] for key in dic.keys() if dic[key] is not None and dic[key]["filter"] > 0.01})

# Using the first row as an example
linha = 2
data = df["keywords"].iloc[linha]
company = df["keywords"].index[linha]

# Create the graph
G = nx.Graph()

# Add nodes to the graph with attributes
for word, attributes in data.items():
    G.add_node(word, **attributes)
    G.add_edge(company, word)

# Node positions
pos = nx.spring_layout(G, seed=42)

# Define concentric circle distances based on recency
date_distances = {
    "level1": 1,  # Most recent nodes
    "level2": 1.5,  # Most recent nodes
    "level3": 2,       # Recently mentioned nodes
    "level4": 2.5,          # Moderately recent
    "level5": 3,     # Moderately recent
    "level8": 1
}

# Lists for node positions and info
node_x = []
node_y = []
node_color = []
node_text = []
node_form = []
node_hovertext = []
custom_data = []
node_size = []
node_distances = []  # Store distances for scaling node positions

# Populate node information
for node in G.nodes:
    if node == company:
        x, y = 0, 0
        node_x.append(x)
        node_y.append(y)
        node_color.append("black")
        if " " in node:
            splitted_text = node.split(" ")
            mid_text = len(splitted_text)//2
            node_text.append(' '.join(splitted_text[:mid_text]) + '<br>' + ' '.join(splitted_text[mid_text:]))
        else:
            node_text.append(node)
        node_form.append("diamond")
        node_hovertext.append(f"Company: {node}")
        custom_data.append("custom data for company allez allez")
        node_size.append(10)
        node_distances.append(0)  # Center node distance
        continue
    
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    if " " in node:
        splitted_text = node.split(" ")
        mid_text = len(splitted_text)//2
        node_text.append(' '.join(splitted_text[:mid_text]) + '<br>' + ' '.join(splitted_text[mid_text:]))
    else:
        node_text.append(node)
    #node_text.append(node)
    node_form.append("circle")
    node_size.append(np.log(G.nodes[node]["weight"])*8)
    
    # Color from red (-1) to green (1)
    sentiment = G.nodes[node]["sentiment"]
    if sentiment < -0.5:
        node_color.append("rgb(204, 0, 0)")
    elif sentiment < -0.05:
        node_color.append("rgb(239, 83, 80)")
    elif sentiment < 0.1:
        node_color.append("rgb(204, 204, 204)")
    elif sentiment < 0.55:
        node_color.append("rgb(102, 187, 106)")
    else:
        node_color.append("rgb(0, 200, 81)")

    last_time_said = max(int(key) for key, value in G.nodes[node]["date"].items() if value is not None)
    node_hovertext.append(
        f"""Word: {node}
        <br>Count: {int(G.nodes[node]['count'])}
        <br>Last said: {str(last_time_said)[:4] + "-" + str(last_time_said)[-2:6]}"""
    )

    # Determine distance category
    years_ago = 202012 - last_time_said
    if True:
        node_distances.append(date_distances["level8"])
    elif years_ago < 99 and random.random() <= 0.4: #ultimo semestre
        node_distances.append(date_distances["level4"])
    elif years_ago < 3 and random.random() <= 0.6: #ultimo semestre
        node_distances.append(date_distances["level2"])
    elif years_ago < 8: #ainda em 2020
        node_distances.append(date_distances["level3"])
    elif years_ago < 99: #2019 e afins
        node_distances.append(date_distances["level4"])
    else: # 2019 e afins
        node_distances.append(date_distances["level5"])

    # SOURCE FOR CUSTOM DATA
    source = G.nodes[node]["source"]
    source_data = ""
    for key in source.keys():
        if source[key] is not None:
            source_data += f"<li>{key}: {int(source[key])}</li>"
    
    # WEBSITES FOR CUSTOM DATA
    websites = G.nodes[node]["news"]
    websites_data = ""
    for website in websites:
        website = website.replace("/wayback/", "/noFrame/replay/")
        websites_data += f"<li><a href='{website}' target='_blank'>{website}</a></li>"

    custom_data.append(f"""
        <h2>Information on {node}:</h2>
        <p>Count: {int(G.nodes[node]['count'])}</p>
        <p>Sentiment: {round(G.nodes[node]["sentiment"], 3)}</p>
        <p>Source:</p>
        <ul>
        {source_data}
        </ul>
        <div id="scrollable-content">
            <ul>
                {websites_data}
            </ul>
        </div>
        <p> iupi </p>
        """)




# Create the Plotly figure
fig = go.Figure()

"""
# Draw concentric circles
for distance in date_distances.values():
    theta = np.linspace(0, 2 * np.pi, 100)
    x_circle = distance * np.cos(theta)
    y_circle = distance * np.sin(theta)
    
    fig.add_trace(
        go.Scatter(
            x=x_circle,
            y=y_circle,
            mode="markers",
            hoverinfo="skip",
            marker=dict(
                color="lightgray",
                size=4
            ),
        )
    )
"""



# Draw nodes
fig.add_trace(
    go.Scatter(
        x=node_x_scaled,
        y=node_y_scaled,
        mode="markers+text",
        text=node_text,
        hovertext=node_hovertext,
        marker=dict(
            color=node_color,
            symbol=node_form,
            size=node_size,
            line=dict(color="black", width=1)
        ),
        hoverinfo="text",
        customdata=custom_data
    )
)

fig.update_layout(
    showlegend=False,
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    margin=dict(l=0, r=0, t=0, b=0)  # Remove all margins
)

# Generate the HTML with full document structure
html_code = fig.to_html(include_plotlyjs='inline', full_html=True)

# Modify the generated HTML to include the side panel and additional styles/scripts
additional_html = """
<style>
    body, html {
        height: 100%;
        margin: 0;
        font-family: Arial, sans-serif;
        overflow: hidden;
    }
    #graph {
        position: absolute;
        top: 0;
        left: 0;
        right: 0;
        bottom: 0;
        z-index: 1; /* Ensure the graph is behind the panel */
    }
    #info-panel {
        position: absolute;
        right: 0; /* Default to right */
        top: 0;
        width: 300px; /* Fixed width of the panel */
        height: 100vh; /* Full height of container */
        background-color: #f4f4f9;
        box-shadow: -2px 0 5px rgba(0,0,0,0.1);
        transform: translateX(100%); /* Hide initially */
        transition: transform 0.3s ease; /* Smooth slide in */
        z-index: 2; /* Panel above the graph */
        display: flex;
        flex-direction: column; /* Arrange children vertically */
    }
    #info-panel.open {
        transform: translateX(0); /* Slide in the panel */
    }
    .close-button {
        background-color: #ff4c4c;
        color: white;
        border: none;
        padding: 10px;
        cursor: pointer;
        float: right;
        margin-bottom: 0px;
    }

    #scrollable-content {
        flex: 1; /* Take remaining vertical space */
        max-height: 200px; /* Adjust height as needed */
        overflow-y: auto; /* Enable vertical scrolling */
        padding: 10px; /* Padding for inner content */
        background-color: #ffffff; /* Background color */
        border: 1px solid #ddd; /* Border for the scrollable area */
        box-shadow: inset 0 0 5px rgba(0,0,0,0.1); /* Inner shadow */
        margin-top: 10px; /* Spacing from the title */
        }
</style>

<div id="info-panel">
    <button class="close-button" onclick="closePanel()">Close</button>
    <p id="node-info">Click a node to view details</p>
</div>

<script>
    function closePanel() {
        var panel = document.getElementById('info-panel');
        panel.classList.remove('open');
        // Manter a posição original do painel
        panel.style.left = ''; // Limpa a posição da esquerda
        panel.style.right = ''; // Limpa a posição da direita
    }

    document.addEventListener('DOMContentLoaded', function() {
        var plotDiv = document.querySelector('.plotly-graph-div');
        
        if (plotDiv) {
            plotDiv.on('plotly_click', function(data) {
                if (data.points.length > 0) {
                    var point = data.points[0];

                    // Accessing custom data
                    var customData = point.customdata; // Access custom data here
                    var additionalInfo = customData; // Format the custom data for display

                    document.getElementById('node-info').innerHTML = additionalInfo; // Use custom data for the panel

                    // Determine mouse click position for panel placement
                    var mouseX = data.event.clientX; // Get mouse X position
                    var panel = document.getElementById('info-panel');

                    // Adjust panel position based on mouse click
                    if (mouseX < window.innerWidth / 2) {
                        panel.style.right = '0'; // Show on the right
                        panel.style.left = 'auto'; // Clear left position
                    } else {
                        panel.style.left = '0'; // Show on the left
                        panel.style.right = 'auto'; // Clear right position
                    }

                    // Show the panel
                    panel.classList.add('open'); // Show panel on click
                }
            });
        } else {
            console.error("Graph div not found.");
        }
    });

</script>
"""

# Combine the Plotly HTML and the additional HTML
final_html = html_code.replace("</body>", additional_html + "</body>")

# Save the HTML file
with open('graph_with_concentric_circles.html', 'w') as f:
    f.write(final_html)

print("HTML file with concentric circles created successfully.")


HTML file with concentric circles created successfully.


In [2]:
import networkx as nx
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import base64
from io import BytesIO
random.seed(42)

# Sample data (Replace this with your actual dataframe)
df = pd.read_parquet('data05.parquet')
df["keywords"] = df["keywords"].map(lambda dic: {key: dic[key] for key in dic.keys() if dic[key] is not None and dic[key]["filter"] > 0.15})

# Using the first row as an example
linha = 2
data = df["keywords"].iloc[linha]
company = df["keywords"].index[linha]

# Create the graph
G = nx.Graph()

# Add nodes to the graph with attributes
for word, attributes in data.items():
    G.add_node(word, **attributes)
    G.add_edge(company, word)

# Node positions
pos = nx.spring_layout(G, seed=42)

# Define concentric circle distances based on recency
date_distances = {
    "level1": 1,  # Most recent nodes
    "level2": 1.5,  # Most recent nodes
    "level3": 2,       # Recently mentioned nodes
    "level4": 2.5,          # Moderately recent
    "level5": 3,     # Moderately recent
    "level8": 1
}

# Lists for node positions and info
node_x = []
node_y = []
node_color = []
node_text = []
node_form = []
node_hovertext = []
custom_data = []
node_size = []
node_distances = []  # Store distances for scaling node positions

# Populate node information
for node in G.nodes:
    if node == company:
        x, y = 0, 0
        node_x.append(x)
        node_y.append(y)
        node_color.append("black")
        if " " in node:
            splitted_text = node.split(" ")
            mid_text = len(splitted_text)//2
            node_text.append(' '.join(splitted_text[:mid_text]) + '<br>' + ' '.join(splitted_text[mid_text:]))
        else:
            node_text.append(node)
        node_form.append("diamond")
        node_hovertext.append(f"Company: {node}")
        custom_data.append("custom data for company allez allez")
        node_size.append(10)
        node_distances.append(0)  # Center node distance
        continue
    
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    if " " in node:
        splitted_text = node.split(" ")
        mid_text = len(splitted_text)//2
        node_text.append(' '.join(splitted_text[:mid_text]) + '<br>' + ' '.join(splitted_text[mid_text:]))
    else:
        node_text.append(node)
    #node_text.append(node)
    node_form.append("circle")
    node_size.append(np.log(G.nodes[node]["weight"])*8)
    
    # Color from red (-1) to green (1)
    sentiment = G.nodes[node]["sentiment"]
    if sentiment < -0.5:
        node_color.append("rgb(204, 0, 0)")
    elif sentiment < -0.05:
        node_color.append("rgb(239, 83, 80)")
    elif sentiment < 0.1:
        node_color.append("rgb(204, 204, 204)")
    elif sentiment < 0.55:
        node_color.append("rgb(102, 187, 106)")
    else:
        node_color.append("rgb(0, 200, 81)")

    last_time_said = max(int(key) for key, value in G.nodes[node]["date"].items() if value is not None)
    node_hovertext.append(
        f"""Word: {node}
        <br>Count: {int(G.nodes[node]['count'])}
        <br>Last said: {str(last_time_said)[:4] + "-" + str(last_time_said)[-2:6]}"""
    )

    # Determine distance category
    years_ago = 202012 - last_time_said
    if True:
        node_distances.append(date_distances["level8"])
    elif years_ago < 99 and random.random() <= 0.4: #ultimo semestre
        node_distances.append(date_distances["level4"])
    elif years_ago < 3 and random.random() <= 0.6: #ultimo semestre
        node_distances.append(date_distances["level2"])
    elif years_ago < 8: #ainda em 2020
        node_distances.append(date_distances["level3"])
    elif years_ago < 99: #2019 e afins
        node_distances.append(date_distances["level4"])
    else: # 2019 e afins
        node_distances.append(date_distances["level5"])

    # SOURCE FOR CUSTOM DATA
    source = G.nodes[node]["source"]
    source_data = ""
    for key in source.keys():
        if source[key] is not None:
            source_data += f"<li>{key}: {int(source[key])}</li>"
    
    # WEBSITES FOR CUSTOM DATA
    websites = G.nodes[node]["news"]
    websites_data = ""
    for website in websites:
        website = website.replace("/wayback/", "/noFrame/replay/")
        websites_data += f"<li><a href='{website}' target='_blank'>{website}</a></li>"

    #PLOT FOR CUSTOM DATA
    categories = ['Apples', 'Bananas', 'Cherries', 'Dates', 'Elderberries']
    values = [10, 15, 7, 5, 20]
    # Create a bar plot
    # Update layout properties
    x = np.arange(1, 11)
    y = np.random.randint(1, 20, size=10)

    plt.figure(figsize=(6, 4))
    plt.bar(x, y, color='blue')
    plt.title('Sample Bar Plot')
    plt.xlabel('X-axis')
    plt.ylabel('Y-axis')
    buffer = BytesIO()
    plt.savefig(buffer, format='png')
    plt.close()  # Close the figure to free memory
    buffer.seek(0)  # Move to the beginning of the BytesIO object
    img_str = base64.b64encode(buffer.read()).decode('utf-8')

    custom_data.append(f"""
        <h2>Information on {node}:</h2>
        <p>Count: {int(G.nodes[node]['count'])}</p>
        <p>Sentiment: {round(G.nodes[node]["sentiment"], 3)}</p>
        <p>Source:</p>
        <ul>
        {source_data}
        </ul>
        <div id="scrollable-content">
            <ul>
                {websites_data}
            </ul>
        </div>
        <img src="data:image/png;base64,{img_str}" alt="Bar Plot" style="width:100%; height:auto;">
        <p> iupi </p>
        """)




# Create the Plotly figure
fig = go.Figure()

"""
# Draw concentric circles
for distance in date_distances.values():
    theta = np.linspace(0, 2 * np.pi, 100)
    x_circle = distance * np.cos(theta)
    y_circle = distance * np.sin(theta)
    
    fig.add_trace(
        go.Scatter(
            x=x_circle,
            y=y_circle,
            mode="markers",
            hoverinfo="skip",
            marker=dict(
                color="lightgray",
                size=4
            ),
        )
    )
"""



# Draw nodes
fig.add_trace(
    go.Scatter(
        x=node_x,
        y=node_y,
        mode="markers+text",
        text=node_text,
        hovertext=node_hovertext,
        marker=dict(
            color=node_color,
            symbol=node_form,
            size=node_size,
            line=dict(color="black", width=1)
        ),
        hoverinfo="text",
        customdata=custom_data
    )
)

fig.update_layout(
    showlegend=False,
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    margin=dict(l=0, r=0, t=0, b=0)  # Remove all margins
)

# Generate the HTML with full document structure
html_code = fig.to_html(include_plotlyjs='inline', full_html=True)

# Modify the generated HTML to include the side panel and additional styles/scripts
additional_html = """
<style>
    body, html {
        height: 100%;
        margin: 0;
        font-family: Arial, sans-serif;
        overflow: hidden;
    }
    #graph {
        position: absolute;
        top: 0;
        left: 0;
        right: 0;
        bottom: 0;
        z-index: 1; /* Ensure the graph is behind the panel */
    }
    #info-panel {
        position: absolute;
        right: 0; /* Default to right */
        top: 0;
        width: 300px; /* Fixed width of the panel */
        height: 100vh; /* Full height of container */
        background-color: #f4f4f9;
        box-shadow: -2px 0 5px rgba(0,0,0,0.1);
        transform: translateX(100%); /* Hide initially */
        transition: transform 0.3s ease; /* Smooth slide in */
        z-index: 2; /* Panel above the graph */
        display: flex;
        flex-direction: column; /* Arrange children vertically */
    }
    #info-panel.open {
        transform: translateX(0); /* Slide in the panel */
    }
    .close-button {
        background-color: #ff4c4c;
        color: white;
        border: none;
        padding: 10px;
        cursor: pointer;
        float: right;
        margin-bottom: 0px;
    }

    #scrollable-content {
        flex: 1; /* Take remaining vertical space */
        max-height: 200px; /* Adjust height as needed */
        overflow-y: auto; /* Enable vertical scrolling */
        padding: 10px; /* Padding for inner content */
        background-color: #ffffff; /* Background color */
        border: 1px solid #ddd; /* Border for the scrollable area */
        box-shadow: inset 0 0 5px rgba(0,0,0,0.1); /* Inner shadow */
        margin-top: 10px; /* Spacing from the title */
        }
</style>

<div id="info-panel">
    <button class="close-button" onclick="closePanel()">Close</button>
    <p id="node-info">Click a node to view details</p>
</div>

<script>
    function closePanel() {
        var panel = document.getElementById('info-panel');
        panel.classList.remove('open');
        // Manter a posição original do painel
        panel.style.left = ''; // Limpa a posição da esquerda
        panel.style.right = ''; // Limpa a posição da direita
    }

    document.addEventListener('DOMContentLoaded', function() {
        var plotDiv = document.querySelector('.plotly-graph-div');
        
        if (plotDiv) {
            plotDiv.on('plotly_click', function(data) {
                if (data.points.length > 0) {
                    var point = data.points[0];

                    // Accessing custom data
                    var customData = point.customdata; // Access custom data here
                    var additionalInfo = customData; // Format the custom data for display

                    document.getElementById('node-info').innerHTML = additionalInfo; // Use custom data for the panel

                    // Determine mouse click position for panel placement
                    var mouseX = data.event.clientX; // Get mouse X position
                    var panel = document.getElementById('info-panel');

                    // Adjust panel position based on mouse click
                    if (mouseX < window.innerWidth / 2) {
                        panel.style.right = '0'; // Show on the right
                        panel.style.left = 'auto'; // Clear left position
                    } else {
                        panel.style.left = '0'; // Show on the left
                        panel.style.right = 'auto'; // Clear right position
                    }

                    // Show the panel
                    panel.classList.add('open'); // Show panel on click
                }
            });
        } else {
            console.error("Graph div not found.");
        }
    });

</script>
"""

# Combine the Plotly HTML and the additional HTML
final_html = html_code.replace("</body>", additional_html + "</body>")

# Save the HTML file
with open('graph_with_concentric_circles.html', 'w') as f:
    f.write(final_html)

print("HTML file with concentric circles created successfully.")


HTML file with concentric circles created successfully.


In [8]:
import networkx as nx
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import base64
from io import BytesIO
random.seed(42)

def rgb_string_to_hex(rgb_string):
    # Remove 'rgb(' and ')' and split the values
    rgb_values = rgb_string.strip('rgb()').split(',')
    # Convert the string values to integers
    rgb = tuple(int(value.strip()) for value in rgb_values)
    # Format to hex
    return '#{:02X}{:02X}{:02X}'.format(rgb[0], rgb[1], rgb[2])

# Sample data (Replace this with your actual dataframe)
df = pd.read_parquet('data05.parquet')
df["keywords"] = df["keywords"].map(lambda dic: {key: dic[key] for key in dic.keys() if dic[key] is not None and dic[key]["filter"] > 0.15})

# Using the first row as an example
linha = 2
data = df["keywords"].iloc[linha]
company = df["keywords"].index[linha]

# Create the graph
G = nx.Graph()

# Add nodes to the graph with attributes
for word, attributes in data.items():
    G.add_node(word, **attributes)
    G.add_edge(company, word)

# Node positions
pos = nx.spring_layout(G, seed=42)

# Define concentric circle distances based on recency
date_distances = {
    "level1": 1,  # Most recent nodes
    "level2": 1.5,  # Most recent nodes
    "level3": 2,       # Recently mentioned nodes
    "level4": 2.5,          # Moderately recent
    "level5": 3,     # Moderately recent
    "level8": 1
}

# Lists for node positions and info
node_x = []
node_y = []
node_color = []
node_text = []
node_form = []
node_hovertext = []
custom_data = []
node_size = []
node_distances = []  # Store distances for scaling node positions

# Populate node information
for node in G.nodes:
    if node == company:
        x, y = 0, 0
        node_x.append(x)
        node_y.append(y)
        node_color.append("black")
        if " " in node:
            splitted_text = node.split(" ")
            mid_text = len(splitted_text)//2
            node_text.append(' '.join(splitted_text[:mid_text]) + '<br>' + ' '.join(splitted_text[mid_text:]))
        else:
            node_text.append(node)
        node_form.append("diamond")
        node_hovertext.append(f"Company: {node}")
        custom_data.append("custom data for company allez allez")
        node_size.append(10)
        node_distances.append(0)  # Center node distance
        continue
    
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    if " " in node:
        splitted_text = node.split(" ")
        mid_text = len(splitted_text)//2
        node_text.append(' '.join(splitted_text[:mid_text]) + '<br>' + ' '.join(splitted_text[mid_text:]))
    else:
        node_text.append(node)
    #node_text.append(node)
    node_form.append("circle")
    node_size.append(np.log(G.nodes[node]["weight"])*8)
    
    # Color from red (-1) to green (1)
    sentiment = G.nodes[node]["sentiment"]
    if sentiment < -0.5:
        node_color.append("rgb(204, 0, 0)")
    elif sentiment < -0.05:
        node_color.append("rgb(239, 83, 80)")
    elif sentiment < 0.1:
        node_color.append("rgb(204, 204, 204)")
    elif sentiment < 0.55:
        node_color.append("rgb(102, 187, 106)")
    else:
        node_color.append("rgb(0, 200, 81)")

    last_time_said = max(int(key) for key, value in G.nodes[node]["date"].items() if value is not None)
    node_hovertext.append(
        f"""Word: {node}
        <br>Count: {int(G.nodes[node]['count'])}
        <br>Last said: {str(last_time_said)[:4] + "-" + str(last_time_said)[-2:6]}"""
    )

    # Determine distance category
    years_ago = 202012 - last_time_said
    if True:
        node_distances.append(date_distances["level8"])
    elif years_ago < 99 and random.random() <= 0.4: #ultimo semestre
        node_distances.append(date_distances["level4"])
    elif years_ago < 3 and random.random() <= 0.6: #ultimo semestre
        node_distances.append(date_distances["level2"])
    elif years_ago < 8: #ainda em 2020
        node_distances.append(date_distances["level3"])
    elif years_ago < 99: #2019 e afins
        node_distances.append(date_distances["level4"])
    else: # 2019 e afins
        node_distances.append(date_distances["level5"])

    # SOURCE FOR CUSTOM DATA
    source = G.nodes[node]["source"]
    source_data = ""
    for key in source.keys():
        if source[key] is not None:
            source_data += f"<li>{key}: {int(source[key])}</li>"
    
    # WEBSITES FOR CUSTOM DATA
    websites = G.nodes[node]["news"]
    websites_data = ""
    for website in websites:
        website = website.replace("/wayback/", "/noFrame/replay/")
        websites_data += f"<li><a href='{website}' target='_blank'>{website}</a></li>"

    #PLOT FOR CUSTOM DATA
    first_time_said = min(int(key) for key, value in G.nodes[node]["date"].items() if value is not None)
    plot_x = []
    plot_y = []
    for key in sorted(G.nodes[node]["date"].keys()):
        if int(key) > int(first_time_said):
            plot_x.append(str(key))
            plot_y.append(int(G.nodes[node]["date"][key]) if G.nodes[node]["date"][key] is not None else 0)

    plt.figure(figsize=(6, 4))
    plt.bar(plot_x, plot_y, color=rgb_string_to_hex(node_color[-1]))
    plt.xlabel('Dates')
    plt.ylabel('Count')
    plt.grid(True, alpha=0.2)
    buffer = BytesIO()
    plt.savefig(buffer, format='png')
    plt.close()  # Close the figure to free memory
    buffer.seek(0)  # Move to the beginning of the BytesIO object
    img_str = base64.b64encode(buffer.read()).decode('utf-8')

    custom_data.append(f"""
        <h2>Information on {node}:</h2>
        <p>Count: {int(G.nodes[node]['count'])}</p>
        <p>Sentiment: {round(G.nodes[node]["sentiment"], 3)}</p>
        <p>Source:</p>
        <ul>
        {source_data}
        </ul>
        <div id="scrollable-content">
            <ul>
                {websites_data}
            </ul>
        </div>
        <img src="data:image/png;base64,{img_str}" alt="Bar Plot" style="width:100%; height:auto;">
        <p> iupi </p>
        """)




# Create the Plotly figure
fig = go.Figure()

"""
# Draw concentric circles
for distance in date_distances.values():
    theta = np.linspace(0, 2 * np.pi, 100)
    x_circle = distance * np.cos(theta)
    y_circle = distance * np.sin(theta)
    
    fig.add_trace(
        go.Scatter(
            x=x_circle,
            y=y_circle,
            mode="markers",
            hoverinfo="skip",
            marker=dict(
                color="lightgray",
                size=4
            ),
        )
    )
"""



# Draw nodes
fig.add_trace(
    go.Scatter(
        x=node_x,
        y=node_y,
        mode="markers+text",
        text=node_text,
        hovertext=node_hovertext,
        marker=dict(
            color=node_color,
            symbol=node_form,
            size=node_size,
            line=dict(color="black", width=1)
        ),
        hoverinfo="text",
        customdata=custom_data
    )
)

fig.update_layout(
    showlegend=False,
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    margin=dict(l=0, r=0, t=0, b=0)  # Remove all margins
)

# Generate the HTML with full document structure
html_code = fig.to_html(include_plotlyjs='inline', full_html=True)

# Modify the generated HTML to include the side panel and additional styles/scripts
additional_html = """
<style>
    body, html {
        height: 100%;
        margin: 0;
        font-family: Arial, sans-serif;
        overflow: hidden;
    }
    #graph {
        position: absolute;
        top: 0;
        left: 0;
        right: 0;
        bottom: 0;
        z-index: 1; /* Ensure the graph is behind the panel */
    }
    #info-panel {
        position: absolute;
        right: 0; /* Default to right */
        top: 0;
        width: 300px; /* Fixed width of the panel */
        height: 100vh; /* Full height of container */
        background-color: #f4f4f9;
        box-shadow: -2px 0 5px rgba(0,0,0,0.1);
        transform: translateX(100%); /* Hide initially */
        transition: transform 0.3s ease; /* Smooth slide in */
        z-index: 2; /* Panel above the graph */
        display: flex;
        flex-direction: column; /* Arrange children vertically */
    }
    #info-panel.open {
        transform: translateX(0); /* Slide in the panel */
    }
    .close-button {
        background-color: #ff4c4c;
        color: white;
        border: none;
        padding: 10px;
        cursor: pointer;
        float: right;
        margin-bottom: 0px;
    }

    #scrollable-content {
        flex: 1; /* Take remaining vertical space */
        max-height: 10px; /* Adjust height as needed */
        overflow-y: auto; /* Enable vertical scrolling */
        padding: 10px; /* Padding for inner content */
        background-color: #ffffff; /* Background color */
        border: 1px solid #ddd; /* Border for the scrollable area */
        box-shadow: inset 0 0 5px rgba(0,0,0,0.1); /* Inner shadow */
        margin-top: 10px; /* Spacing from the title */
        }
</style>

<div id="info-panel">
    <button class="close-button" onclick="closePanel()">Close</button>
    <p id="node-info">Click a node to view details</p>
</div>

<script>
    function closePanel() {
        var panel = document.getElementById('info-panel');
        panel.classList.remove('open');
        // Manter a posição original do painel
        panel.style.left = ''; // Limpa a posição da esquerda
        panel.style.right = ''; // Limpa a posição da direita
    }

    document.addEventListener('DOMContentLoaded', function() {
        var plotDiv = document.querySelector('.plotly-graph-div');
        
        if (plotDiv) {
            plotDiv.on('plotly_click', function(data) {
                if (data.points.length > 0) {
                    var point = data.points[0];

                    // Accessing custom data
                    var customData = point.customdata; // Access custom data here
                    var additionalInfo = customData; // Format the custom data for display

                    document.getElementById('node-info').innerHTML = additionalInfo; // Use custom data for the panel

                    // Determine mouse click position for panel placement
                    var mouseX = data.event.clientX; // Get mouse X position
                    var panel = document.getElementById('info-panel');

                    // Adjust panel position based on mouse click
                    if (mouseX < window.innerWidth / 2) {
                        panel.style.right = '0'; // Show on the right
                        panel.style.left = 'auto'; // Clear left position
                    } else {
                        panel.style.left = '0'; // Show on the left
                        panel.style.right = 'auto'; // Clear right position
                    }

                    // Show the panel
                    panel.classList.add('open'); // Show panel on click
                }
            });
        } else {
            console.error("Graph div not found.");
        }
    });

</script>
"""

# Combine the Plotly HTML and the additional HTML
final_html = html_code.replace("</body>", additional_html + "</body>")

# Save the HTML file
with open('graph_with_concentric_circles.html', 'w') as f:
    f.write(final_html)

print("HTML file with concentric circles created successfully.")


HTML file with concentric circles created successfully.


In [ ]:
import networkx as nx
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import base64
from io import BytesIO
random.seed(42)

def rgb_string_to_hex(rgb_string):
    # Remove 'rgb(' and ')' and split the values
    rgb_values = rgb_string.strip('rgb()').split(',')
    # Convert the string values to integers
    rgb = tuple(int(value.strip()) for value in rgb_values)
    # Format to hex
    return '#{:02X}{:02X}{:02X}'.format(rgb[0], rgb[1], rgb[2])

# Sample data (Replace this with your actual dataframe)
df = pd.read_parquet('data05.parquet')
df["keywords"] = df["keywords"].map(lambda dic: {key: dic[key] for key in dic.keys() if dic[key] is not None and dic[key]["filter"] > 0.15})

# Using the first row as an example
linha = 2
data = df["keywords"].iloc[linha]
company = df["keywords"].index[linha]

# Create the graph
G = nx.Graph()

# Add nodes to the graph with attributes
for word, attributes in data.items():
    G.add_node(word, **attributes)
    G.add_edge(company, word)

# Node positions
pos = nx.spring_layout(G, seed=42)

# Define concentric circle distances based on recency
date_distances = {
    "level1": 1,  # Most recent nodes
    "level2": 1.5,  # Most recent nodes
    "level3": 2,       # Recently mentioned nodes
    "level4": 2.5,          # Moderately recent
    "level5": 3,     # Moderately recent
    "level8": 1
}

# Lists for node positions and info
node_x = []
node_y = []
node_color = []
node_text = []
node_form = []
node_hovertext = []
custom_data = []
node_size = []
node_distances = []  # Store distances for scaling node positions

# Populate node information
for node in G.nodes:
    if node == company:
        x, y = 0, 0
        node_x.append(x)
        node_y.append(y)
        node_color.append("black")
        if " " in node:
            splitted_text = node.split(" ")
            mid_text = len(splitted_text)//2
            node_text.append(' '.join(splitted_text[:mid_text]) + '<br>' + ' '.join(splitted_text[mid_text:]))
        else:
            node_text.append(node)
        node_form.append("diamond")
        node_hovertext.append(f"Company: {node}")
        custom_data.append("custom data for company allez allez")
        node_size.append(10)
        node_distances.append(0)  # Center node distance
        continue
    
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    if " " in node:
        splitted_text = node.split(" ")
        mid_text = len(splitted_text)//2
        node_text.append(' '.join(splitted_text[:mid_text]) + '<br>' + ' '.join(splitted_text[mid_text:]))
    else:
        node_text.append(node)
    #node_text.append(node)
    node_form.append("circle")
    node_size.append(np.log(G.nodes[node]["weight"])*8)
    
    # Color from red (-1) to green (1)
    sentiment = G.nodes[node]["sentiment"]
    if sentiment < -0.5:
        node_color.append("rgb(204, 0, 0)")
    elif sentiment < -0.05:
        node_color.append("rgb(239, 83, 80)")
    elif sentiment < 0.1:
        node_color.append("rgb(204, 204, 204)")
    elif sentiment < 0.55:
        node_color.append("rgb(102, 187, 106)")
    else:
        node_color.append("rgb(0, 200, 81)")

    last_time_said = max(int(key) for key, value in G.nodes[node]["date"].items() if value is not None)
    node_hovertext.append(
        f"""Word: {node}
        <br>Count: {int(G.nodes[node]['count'])}
        <br>Last said: {str(last_time_said)[:4] + "-" + str(last_time_said)[-2:6]}"""
    )

    # Determine distance category
    years_ago = 202012 - last_time_said
    if True:
        node_distances.append(date_distances["level8"])
    elif years_ago < 99 and random.random() <= 0.4: #ultimo semestre
        node_distances.append(date_distances["level4"])
    elif years_ago < 3 and random.random() <= 0.6: #ultimo semestre
        node_distances.append(date_distances["level2"])
    elif years_ago < 8: #ainda em 2020
        node_distances.append(date_distances["level3"])
    elif years_ago < 99: #2019 e afins
        node_distances.append(date_distances["level4"])
    else: # 2019 e afins
        node_distances.append(date_distances["level5"])

    # SOURCE FOR CUSTOM DATA
    source = G.nodes[node]["source"]
    source_data = ""
    for key in source.keys():
        if source[key] is not None:
            source_data += f"<li>{key}: {int(source[key])}</li>"
    
    # WEBSITES FOR CUSTOM DATA
    websites = G.nodes[node]["news"]
    websites_data = ""
    for website in websites:
        website = website.replace("/wayback/", "/noFrame/replay/")
        websites_data += f"<li><a href='{website}' target='_blank'>{website}</a></li>"

    #PLOT FOR CUSTOM DATA
    first_time_said = min(int(key) for key, value in G.nodes[node]["date"].items() if value is not None)
    times_said_by_year = {}
    for key in sorted(G.nodes[node]["date"].keys()):
        if int(key) >= int(first_time_said):
            times_said = int(G.nodes[node]["date"][key]) if G.nodes[node]["date"][key] is not None else 0
            if key[2:4] not in times_said_by_year:
                times_said_by_year[key[2:4]] = times_said
            else:
                times_said_by_year[key[2:4]] += times_said

    plt.figure(figsize=(6, 4))
    plt.bar(times_said_by_year.keys(), times_said_by_year.values(), color=rgb_string_to_hex(node_color[-1]))
    plt.xlabel('Years (2000)')
    plt.ylabel('Number of Mentions')
    plt.grid(axis='y', alpha=0.2)
    buffer = BytesIO()
    plt.savefig(buffer, format='png', transparent=True)
    plt.close()  # Close the figure to free memory
    buffer.seek(0)  # Move to the beginning of the BytesIO object
    img_str = base64.b64encode(buffer.read()).decode('utf-8')

    custom_data.append(f"""
        <h2>Information on {node}:</h2>
        <p>Count: {int(G.nodes[node]['count'])}</p>
        <p>Sentiment: {round(G.nodes[node]["sentiment"], 3)}</p>
        <p>Source:</p>
        <ul>
        {source_data}
        </ul>
        <div id="scrollable-content">
            <ul>
                {websites_data}
            </ul>
        </div>
        <img src="data:image/png;base64,{img_str}" alt="Bar Plot" style="width:100%; height:auto;">
        <p> iupi </p>
        """)




# Create the Plotly figure
fig = go.Figure()

"""
# Draw concentric circles
for distance in date_distances.values():
    theta = np.linspace(0, 2 * np.pi, 100)
    x_circle = distance * np.cos(theta)
    y_circle = distance * np.sin(theta)
    
    fig.add_trace(
        go.Scatter(
            x=x_circle,
            y=y_circle,
            mode="markers",
            hoverinfo="skip",
            marker=dict(
                color="lightgray",
                size=4
            ),
        )
    )
"""



# Draw nodes
fig.add_trace(
    go.Scatter(
        x=node_x,
        y=node_y,
        mode="markers+text",
        text=node_text,
        hovertext=node_hovertext,
        marker=dict(
            color=node_color,
            symbol=node_form,
            size=node_size,
            line=dict(color="black", width=1)
        ),
        hoverinfo="text",
        customdata=custom_data
    )
)

fig.update_layout(
    showlegend=False,
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    margin=dict(l=0, r=0, t=0, b=0)  # Remove all margins
)

# Generate the HTML with full document structure
html_code = fig.to_html(include_plotlyjs='inline', full_html=True)

# Modify the generated HTML to include the side panel and additional styles/scripts
additional_html = """
<style>
    body, html {
        height: 100%;
        margin: 0;
        font-family: Arial, sans-serif;
        overflow: hidden;
    }
    #graph {
        position: absolute;
        top: 0;
        left: 0;
        right: 0;
        bottom: 0;
        z-index: 1; /* Ensure the graph is behind the panel */
    }
    #info-panel {
        position: absolute;
        right: 0; /* Default to right */
        top: 0;
        width: 300px; /* Fixed width of the panel */
        height: 100vh; /* Full height of container */
        background-color: #f4f4f9;
        box-shadow: -2px 0 5px rgba(0,0,0,0.1);
        transform: translateX(100%); /* Hide initially */
        transition: transform 0.3s ease; /* Smooth slide in */
        z-index: 2; /* Panel above the graph */
        display: flex;
        flex-direction: column; /* Arrange children vertically */
    }
    #info-panel.open {
        transform: translateX(0); /* Slide in the panel */
    }
    .close-button {
        background-color: #ff4c4c;
        color: white;
        border: none;
        padding: 10px;
        cursor: pointer;
        float: right;
        margin-bottom: 0px;
    }

    #scrollable-content {
        flex: 1; /* Take remaining vertical space */
        max-height: 25px; /* Adjust height as needed */
        overflow-y: auto; /* Enable vertical scrolling */
        padding: 10px; /* Padding for inner content */
        background-color: #ffffff; /* Background color */
        border: 1px solid #ddd; /* Border for the scrollable area */
        box-shadow: inset 0 0 5px rgba(0,0,0,0.1); /* Inner shadow */
        margin-top: 10px; /* Spacing from the title */
        }
</style>

<div id="info-panel">
    <button class="close-button" onclick="closePanel()">Close</button>
    <p id="node-info">Click a node to view details</p>
</div>

<script>
    function closePanel() {
        var panel = document.getElementById('info-panel');
        panel.classList.remove('open');
        // Manter a posição original do painel
        panel.style.left = ''; // Limpa a posição da esquerda
        panel.style.right = ''; // Limpa a posição da direita
    }

    document.addEventListener('DOMContentLoaded', function() {
        var plotDiv = document.querySelector('.plotly-graph-div');
        
        if (plotDiv) {
            plotDiv.on('plotly_click', function(data) {
                if (data.points.length > 0) {
                    var point = data.points[0];

                    // Accessing custom data
                    var customData = point.customdata; // Access custom data here
                    var additionalInfo = customData; // Format the custom data for display

                    document.getElementById('node-info').innerHTML = additionalInfo; // Use custom data for the panel

                    // Determine mouse click position for panel placement
                    var mouseX = data.event.clientX; // Get mouse X position
                    var panel = document.getElementById('info-panel');

                    // Adjust panel position based on mouse click
                    if (mouseX < window.innerWidth / 2) {
                        panel.style.right = '0'; // Show on the right
                        panel.style.left = 'auto'; // Clear left position
                    } else {
                        panel.style.left = '0'; // Show on the left
                        panel.style.right = 'auto'; // Clear right position
                    }

                    // Show the panel
                    panel.classList.add('open'); // Show panel on click
                }
            });
        } else {
            console.error("Graph div not found.");
        }
    });

</script>
"""

# Combine the Plotly HTML and the additional HTML
final_html = html_code.replace("</body>", additional_html + "</body>")

# Save the HTML file
with open('graph_with_concentric_circles.html', 'w') as f:
    f.write(final_html)

print("HTML file with concentric circles created successfully.")


HTML file with concentric circles created successfully.


In [24]:
import networkx as nx
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import base64
from io import BytesIO
random.seed(42)

def rgb_string_to_hex(rgb_string):
    # Remove 'rgb(' and ')' and split the values
    rgb_values = rgb_string.strip('rgb()').split(',')
    # Convert the string values to integers
    rgb = tuple(int(value.strip()) for value in rgb_values)
    # Format to hex
    return '#{:02X}{:02X}{:02X}'.format(rgb[0], rgb[1], rgb[2])

# Sample data (Replace this with your actual dataframe)
df = pd.read_parquet('data05.parquet')
df["keywords"] = df["keywords"].map(lambda dic: {key: dic[key] for key in dic.keys() if dic[key] is not None and dic[key]["filter"] > 0.15})

# Using the first row as an example
linha = 2
data = df["keywords"].iloc[linha]
company = df["keywords"].index[linha]

# Create the graph
G = nx.Graph()

# Add nodes to the graph with attributes
for word, attributes in data.items():
    G.add_node(word, **attributes)
    G.add_edge(company, word)

# Node positions
pos = nx.spring_layout(G, seed=42)

# Define concentric circle distances based on recency
date_distances = {
    "level1": 1,  # Most recent nodes
    "level2": 1.5,  # Most recent nodes
    "level3": 2,       # Recently mentioned nodes
    "level4": 2.5,          # Moderately recent
    "level5": 3,     # Moderately recent
    "level8": 1
}

# Lists for node positions and info
node_x = []
node_y = []
node_color = []
node_text = []
node_form = []
node_hovertext = []
custom_data = []
node_size = []
node_distances = []  # Store distances for scaling node positions

# Populate node information
for node in G.nodes:
    if node == company:
        x, y = 0, 0
        node_x.append(x)
        node_y.append(y)
        node_color.append("black")
        if " " in node:
            splitted_text = node.split(" ")
            mid_text = len(splitted_text)//2
            node_text.append(' '.join(splitted_text[:mid_text]) + '<br>' + ' '.join(splitted_text[mid_text:]))
        else:
            node_text.append(node)
        node_form.append("diamond")
        node_hovertext.append(f"Company: {node}")
        custom_data.append("custom data for company allez allez")
        node_size.append(10)
        node_distances.append(0)  # Center node distance
        continue
    
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    if " " in node:
        splitted_text = node.split(" ")
        mid_text = len(splitted_text)//2
        node_text.append(' '.join(splitted_text[:mid_text]) + '<br>' + ' '.join(splitted_text[mid_text:]))
    else:
        node_text.append(node)
    #node_text.append(node)
    node_form.append("circle")
    node_size.append(np.log(G.nodes[node]["weight"])*8)
    
    # Color from red (-1) to green (1)
    sentiment = G.nodes[node]["sentiment"]
    if sentiment < -0.5:
        node_color.append("rgb(204, 0, 0)")
    elif sentiment < -0.05:
        node_color.append("rgb(239, 83, 80)")
    elif sentiment < 0.1:
        node_color.append("rgb(204, 204, 204)")
    elif sentiment < 0.55:
        node_color.append("rgb(102, 187, 106)")
    else:
        node_color.append("rgb(0, 200, 81)")

    last_time_said = max(int(key) for key, value in G.nodes[node]["date"].items() if value is not None)
    node_hovertext.append(
        f"""Word: {node}
        <br>Count: {int(G.nodes[node]['count'])}
        <br>Last said: {str(last_time_said)[:4] + "-" + str(last_time_said)[-2:6]}"""
    )

    # Determine distance category
    years_ago = 202012 - last_time_said
    if True:
        node_distances.append(date_distances["level8"])
    elif years_ago < 99 and random.random() <= 0.4: #ultimo semestre
        node_distances.append(date_distances["level4"])
    elif years_ago < 3 and random.random() <= 0.6: #ultimo semestre
        node_distances.append(date_distances["level2"])
    elif years_ago < 8: #ainda em 2020
        node_distances.append(date_distances["level3"])
    elif years_ago < 99: #2019 e afins
        node_distances.append(date_distances["level4"])
    else: # 2019 e afins
        node_distances.append(date_distances["level5"])

    # SOURCE FOR CUSTOM DATA
    source = G.nodes[node]["source"]
    source_data = ""
    for key in source.keys():
        if source[key] is not None:
            source_data += f"<li>{key}: {int(source[key])}</li>"
    
    # WEBSITES FOR CUSTOM DATA
    websites = sorted(G.nodes[node]["news"], reverse=True)
    websites_data = ""
    for website in websites:
        website = website.replace("/wayback/", "/noFrame/replay/")
        websites_data += f"<li><a href='{website}' target='_blank'>{website[34:38]+'/'+website[38:40] + ' - ' + website[57:]}</a></li>"

    #PLOT FOR CUSTOM DATA
    first_time_said = min(int(key) for key, value in G.nodes[node]["date"].items() if value is not None)
    times_said_by_year = {}
    for key in sorted(G.nodes[node]["date"].keys()):
        if int(key) >= int(first_time_said):
            times_said = int(G.nodes[node]["date"][key]) if G.nodes[node]["date"][key] is not None else 0
            if key[2:4] not in times_said_by_year:
                times_said_by_year[key[2:4]] = times_said
            else:
                times_said_by_year[key[2:4]] += times_said
    plt.figure(figsize=(6, 4))
    plt.bar(times_said_by_year.keys(), times_said_by_year.values(), color=rgb_string_to_hex(node_color[-1]))
    plt.xlabel('Years (2000)')
    plt.ylabel('Number of Mentions')
    plt.grid(axis='y', alpha=0.2)
    buffer = BytesIO()
    plt.savefig(buffer, format='png', transparent=True)
    plt.close() 
    buffer.seek(0)
    img_str = base64.b64encode(buffer.read()).decode('utf-8')

    custom_data.append(f"""
        <h2>Information on {node}:</h2>
        <p>Count: {int(G.nodes[node]['count'])}</p>
        <p>Sentiment: {round(G.nodes[node]["sentiment"], 3)}</p>
        <p>Source:</p>
        <ul>
        {source_data}
        </ul>
        <div id="scrollable-content">
            <ul>
                {websites_data}
            </ul>
        </div>
        <img src="data:image/png;base64,{img_str}" alt="Bar Plot" style="width:100%; height:auto;">
        <p> iupi </p>
        """)




# Create the Plotly figure
fig = go.Figure()

"""
# Draw concentric circles
for distance in date_distances.values():
    theta = np.linspace(0, 2 * np.pi, 100)
    x_circle = distance * np.cos(theta)
    y_circle = distance * np.sin(theta)
    
    fig.add_trace(
        go.Scatter(
            x=x_circle,
            y=y_circle,
            mode="markers",
            hoverinfo="skip",
            marker=dict(
                color="lightgray",
                size=4
            ),
        )
    )
"""



# Draw nodes
fig.add_trace(
    go.Scatter(
        x=node_x,
        y=node_y,
        mode="markers+text",
        text=node_text,
        hovertext=node_hovertext,
        marker=dict(
            color=node_color,
            symbol=node_form,
            size=node_size,
            line=dict(color="black", width=1)
        ),
        hoverinfo="text",
        customdata=custom_data
    )
)

fig.update_layout(
    showlegend=False,
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    margin=dict(l=0, r=0, t=0, b=0)  # Remove all margins
)

# Generate the HTML with full document structure
html_code = fig.to_html(include_plotlyjs='inline', full_html=True)

# Modify the generated HTML to include the side panel and additional styles/scripts
additional_html = """
<style>
    body, html {
        height: 100%;
        margin: 0;
        font-family: Arial, sans-serif;
        overflow: hidden;
    }
    #graph {
        position: absolute;
        top: 0;
        left: 0;
        right: 0;
        bottom: 0;
        z-index: 1; /* Ensure the graph is behind the panel */
    }
    #info-panel {
        position: absolute;
        right: 0; /* Default to right */
        top: 0;
        width: 300px; /* Fixed width of the panel */
        height: 100vh; /* Full height of container */
        background-color: #f4f4f9;
        box-shadow: -2px 0 5px rgba(0,0,0,0.1);
        transform: translateX(100%); /* Hide initially */
        transition: transform 0.3s ease; /* Smooth slide in */
        z-index: 2; /* Panel above the graph */
        display: flex;
        flex-direction: column; /* Arrange children vertically */
    }
    #info-panel.open {
        transform: translateX(0); /* Slide in the panel */
    }
    .close-button {
        background-color: #ff4c4c;
        color: white;
        border: none;
        padding: 10px;
        cursor: pointer;
        float: right;
        margin-bottom: 0px;
    }

    #scrollable-content {
        flex: 1; /* Take remaining vertical space */
        max-height: 25px; /* Adjust height as needed */
        overflow-y: auto; /* Enable vertical scrolling */
        overflow-x: auto;
        padding: 4px; /* Padding for inner content */
        background-color: #ffffff; /* Background color */
        border: 1px solid #ddd; /* Border for the scrollable area */
        box-shadow: inset 0 0 5px rgba(0,0,0,0.1); /* Inner shadow */
        margin-top: 10px; /* Spacing from the title */
        }
    
    #scrollable-content li {
    white-space: nowrap; /* Prevents line breaks within the item */
    margin-right: 20px; /* Adds space between items */
    }
</style>

<div id="info-panel">
    <button class="close-button" onclick="closePanel()">Close</button>
    <p id="node-info">Click a node to view details</p>
</div>

<script>
    function closePanel() {
        var panel = document.getElementById('info-panel');
        panel.classList.remove('open');
        // Manter a posição original do painel
        panel.style.left = ''; // Limpa a posição da esquerda
        panel.style.right = ''; // Limpa a posição da direita
    }

    document.addEventListener('DOMContentLoaded', function() {
        var plotDiv = document.querySelector('.plotly-graph-div');
        
        if (plotDiv) {
            plotDiv.on('plotly_click', function(data) {
                if (data.points.length > 0) {
                    var point = data.points[0];

                    // Accessing custom data
                    var customData = point.customdata; // Access custom data here
                    var additionalInfo = customData; // Format the custom data for display

                    document.getElementById('node-info').innerHTML = additionalInfo; // Use custom data for the panel

                    // Determine mouse click position for panel placement
                    var mouseX = data.event.clientX; // Get mouse X position
                    var panel = document.getElementById('info-panel');

                    // Adjust panel position based on mouse click
                    if (mouseX < window.innerWidth / 2) {
                        panel.style.right = '0'; // Show on the right
                        panel.style.left = 'auto'; // Clear left position
                    } else {
                        panel.style.left = '0'; // Show on the left
                        panel.style.right = 'auto'; // Clear right position
                    }

                    // Show the panel
                    panel.classList.add('open'); // Show panel on click
                }
            });
        } else {
            console.error("Graph div not found.");
        }
    });

</script>
"""

# Combine the Plotly HTML and the additional HTML
final_html = html_code.replace("</body>", additional_html + "</body>")

# Save the HTML file
with open('graph_with_concentric_circles.html', 'w') as f:
    f.write(final_html)

print("HTML file with concentric circles created successfully.")


HTML file with concentric circles created successfully.


In [1]:
import networkx as nx
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import base64
from io import BytesIO
random.seed(42)

def rgb_string_to_hex(rgb_string):
    # Remove 'rgb(' and ')' and split the values
    rgb_values = rgb_string.strip('rgb()').split(',')
    # Convert the string values to integers
    rgb = tuple(int(value.strip()) for value in rgb_values)
    # Format to hex
    return '#{:02X}{:02X}{:02X}'.format(rgb[0], rgb[1], rgb[2])

# Sample data (Replace this with your actual dataframe)
df = pd.read_parquet('data05.parquet')
df["keywords"] = df["keywords"].map(lambda dic: {key: dic[key] for key in dic.keys() if dic[key] is not None and dic[key]["filter"] > 0.15})

# Using the first row as an example
linha = 2
data = df["keywords"].iloc[linha]
company = df["keywords"].index[linha]

# Create the graph
G = nx.Graph()

# Add nodes to the graph with attributes
for word, attributes in data.items():
    G.add_node(word, **attributes)
    G.add_edge(company, word)

# Node positions
pos = nx.spring_layout(G, seed=42)

# Define concentric circle distances based on recency
date_distances = {
    "level1": 1,  # Most recent nodes
    "level2": 1.5,  # Most recent nodes
    "level3": 2,       # Recently mentioned nodes
    "level4": 2.5,          # Moderately recent
    "level5": 3,     # Moderately recent
    "level8": 1
}

# Lists for node positions and info
node_x = []
node_y = []
node_color = []
node_text = []
node_form = []
node_hovertext = []
custom_data = []
node_size = []
node_distances = []  # Store distances for scaling node positions

# Populate node information
for node in G.nodes:
    if node == company:
        x, y = 0, 0
        node_x.append(x)
        node_y.append(y)
        node_color.append("black")
        if " " in node:
            splitted_text = node.split(" ")
            mid_text = len(splitted_text)//2
            node_text.append(' '.join(splitted_text[:mid_text]) + '<br>' + ' '.join(splitted_text[mid_text:]))
        else:
            node_text.append(node)
        node_form.append("diamond")
        node_hovertext.append(f"Company: {node}")
        custom_data.append("custom data for company allez allez")
        node_size.append(10)
        node_distances.append(0)  # Center node distance
        continue
    
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    if " " in node:
        splitted_text = node.split(" ")
        mid_text = len(splitted_text)//2
        node_text.append(' '.join(splitted_text[:mid_text]) + '<br>' + ' '.join(splitted_text[mid_text:]))
    else:
        node_text.append(node)
    #node_text.append(node)
    node_form.append("circle")
    node_size.append(np.log(G.nodes[node]["weight"])*8)
    
    # Color from red (-1) to green (1)
    sentiment = G.nodes[node]["sentiment"]
    if sentiment < -0.5:
        node_color.append("rgb(204, 0, 0)")
    elif sentiment < -0.05:
        node_color.append("rgb(239, 83, 80)")
    elif sentiment < 0.1:
        node_color.append("rgb(204, 204, 204)")
    elif sentiment < 0.55:
        node_color.append("rgb(102, 187, 106)")
    else:
        node_color.append("rgb(0, 200, 81)")

    last_time_said = max(int(key) for key, value in G.nodes[node]["date"].items() if value is not None)
    node_hovertext.append(
        f"""Word: {node}
        <br>Count: {int(G.nodes[node]['count'])}
        <br>Last said: {str(last_time_said)[:4] + "-" + str(last_time_said)[-2:6]}"""
    )

    # Determine distance category
    years_ago = 202012 - last_time_said
    if True:
        node_distances.append(date_distances["level8"])
    elif years_ago < 99 and random.random() <= 0.4: #ultimo semestre
        node_distances.append(date_distances["level4"])
    elif years_ago < 3 and random.random() <= 0.6: #ultimo semestre
        node_distances.append(date_distances["level2"])
    elif years_ago < 8: #ainda em 2020
        node_distances.append(date_distances["level3"])
    elif years_ago < 99: #2019 e afins
        node_distances.append(date_distances["level4"])
    else: # 2019 e afins
        node_distances.append(date_distances["level5"])

    # SOURCE FOR CUSTOM DATA
    source = G.nodes[node]["source"]
    source_data = ""
    for key in source.keys():
        if source[key] is not None:
            source_data += f"<li>{key}: {int(source[key])}</li>"
    
    # WEBSITES FOR CUSTOM DATA
    websites = sorted(G.nodes[node]["news"], reverse=True)
    websites_data = ""
    for website in websites:
        website = website.replace("/wayback/", "/noFrame/replay/")
        websites_data += f"<li><a href='{website}' target='_blank'>{website[34:38]+'/'+website[38:40] + ' - ' + website[57:]}</a></li>"

    #PLOT FOR CUSTOM DATA
    first_time_said = min(int(key) for key, value in G.nodes[node]["date"].items() if value is not None)
    times_said_by_year = {}
    for key in sorted(G.nodes[node]["date"].keys()):
        if int(key) >= int(first_time_said):
            times_said = int(G.nodes[node]["date"][key]) if G.nodes[node]["date"][key] is not None else 0
            if key[2:4] not in times_said_by_year:
                times_said_by_year[key[2:4]] = times_said
            else:
                times_said_by_year[key[2:4]] += times_said
    plt.figure(figsize=(6, 4))
    plt.bar(times_said_by_year.keys(), times_said_by_year.values(), color=rgb_string_to_hex(node_color[-1]))
    plt.xlabel('Years (2000)')
    plt.ylabel('Number of Mentions')
    plt.grid(axis='y', alpha=0.2)
    buffer = BytesIO()
    plt.savefig(buffer, format='png', transparent=True)
    plt.close() 
    buffer.seek(0)
    img_str = base64.b64encode(buffer.read()).decode('utf-8')

    custom_data.append(f"""
        <h2>Information on {node}:</h2>
        <p>Count: {int(G.nodes[node]['count'])}</p>
        <p>Sentiment: {round(G.nodes[node]["sentiment"], 3)}</p>
        <p>Source:</p>
        <ul>
        {source_data}
        </ul>
        <div id="scrollable-content">
            <ul>
                {websites_data}
            </ul>
        </div>
        <img src="data:image/png;base64,{img_str}" alt="Bar Plot" style="width:100%; height:auto;">
        <p> iupi </p>
        """)




# Create the Plotly figure
fig = go.Figure()

"""
# Draw concentric circles
for distance in date_distances.values():
    theta = np.linspace(0, 2 * np.pi, 100)
    x_circle = distance * np.cos(theta)
    y_circle = distance * np.sin(theta)
    
    fig.add_trace(
        go.Scatter(
            x=x_circle,
            y=y_circle,
            mode="markers",
            hoverinfo="skip",
            marker=dict(
                color="lightgray",
                size=4
            ),
        )
    )
"""



# Draw nodes
fig.add_trace(
    go.Scatter(
        x=node_x,
        y=node_y,
        mode="markers+text",
        text=node_text,
        hovertext=node_hovertext,
        marker=dict(
            color=node_color,
            symbol=node_form,
            size=node_size,
            line=dict(color="black", width=1)
        ),
        hoverinfo="text",
        customdata=custom_data
    )
)

fig.update_layout(
    showlegend=False,
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    margin=dict(l=0, r=0, t=0, b=0)  # Remove all margins
)

# Generate the HTML with full document structure
html_code = fig.to_html(include_plotlyjs='inline', full_html=True)

# Modify the generated HTML to include the side panel and additional styles/scripts
additional_html = """
<style>
    body, html {
        height: 100%;
        margin: 0;
        font-family: Arial, sans-serif;
        overflow: hidden;
    }
    #graph {
        position: absolute;
        top: 0;
        left: 0;
        right: 0;
        bottom: 0;
        z-index: 1; /* Ensure the graph is behind the panel */
    }
    #info-panel {
        position: absolute;
        right: 0; /* Default to right */
        top: 0;
        width: 300px; /* Fixed width of the panel */
        height: 100vh; /* Full height of container */
        background-color: #f4f4f9;
        box-shadow: -2px 0 5px rgba(0,0,0,0.1);
        transform: translateX(100%); /* Hide initially */
        transition: transform 0.3s ease; /* Smooth slide in */
        z-index: 2; /* Panel above the graph */
        display: flex;
        flex-direction: column; /* Arrange children vertically */
    }
    #info-panel.open {
        transform: translateX(0); /* Slide in the panel */
    }
    .close-button {
        background-color: #ff4c4c;
        color: white;
        border: none;
        padding: 10px;
        cursor: pointer;
        float: right;
        margin-bottom: 0px;
    }

    #scrollable-content {
        flex: 1; /* Take remaining vertical space */
        max-height: 25px; /* Adjust height as needed */
        overflow-y: auto; /* Enable vertical scrolling */
        overflow-x: auto;
        padding: 4px; /* Padding for inner content */
        background-color: #ffffff; /* Background color */
        border: 1px solid #ddd; /* Border for the scrollable area */
        box-shadow: inset 0 0 5px rgba(0,0,0,0.1); /* Inner shadow */
        margin-top: 10px; /* Spacing from the title */
        }
    
    #scrollable-content li {
    white-space: nowrap; /* Prevents line breaks within the item */
    margin-right: 20px; /* Adds space between items */
    }
</style>

<div id="info-panel">
    <button class="close-button" onclick="closePanel()">Close</button>
    <p id="node-info">Click a node to view details</p>
</div>

<script>
    function closePanel() {
        var panel = document.getElementById('info-panel');
        panel.classList.remove('open');
        // Manter a posição original do painel
        panel.style.left = ''; // Limpa a posição da esquerda
        panel.style.right = ''; // Limpa a posição da direita
    }

    document.addEventListener('DOMContentLoaded', function() {
        var plotDiv = document.querySelector('.plotly-graph-div');
        
        if (plotDiv) {
            plotDiv.on('plotly_click', function(data) {
                if (data.points.length > 0) {
                    var point = data.points[0];

                    // Accessing custom data
                    var customData = point.customdata; // Access custom data here
                    var additionalInfo = customData; // Format the custom data for display

                    document.getElementById('node-info').innerHTML = additionalInfo; // Use custom data for the panel

                    // Determine mouse click position for panel placement
                    var mouseX = data.event.clientX; // Get mouse X position
                    var panel = document.getElementById('info-panel');

                    // Adjust panel position based on mouse click
                    if (mouseX < window.innerWidth / 2) {
                        panel.style.right = '0'; // Show on the right
                        panel.style.left = 'auto'; // Clear left position
                    } else {
                        panel.style.left = '0'; // Show on the left
                        panel.style.right = 'auto'; // Clear right position
                    }

                    // Show the panel
                    panel.classList.add('open'); // Show panel on click
                }
            });
        } else {
            console.error("Graph div not found.");
        }
    });

</script>
"""

# Combine the Plotly HTML and the additional HTML
final_html = html_code.replace("</body>", additional_html + "</body>")

# Save the HTML file
with open('graph_with_concentric_circles.html', 'w') as f:
    f.write(final_html)

print("HTML file with concentric circles created successfully.")


HTML file with concentric circles created successfully.
